In [1]:
import os
import pandas as pd
import json

netflix_file = os.path.join('Resources','netflix_titles.csv')
metadata_file = os.path.join('Resources','movies_metadata.csv')
credit_file = os.path.join('Resources','credits.csv')

In [2]:
def dictify(record):
    '''
    Returns a dict version of the complex record file in the credit_df
    '''
    dict_val = record
    dict_val = (dict_val.strip('\"'))

    dict_val = (dict_val.strip('['))
    dict_val = (dict_val.strip(']'))
    try:
        dict_out = eval(dict_val)    
    except:
        dict_out = tuple(({'N/A':'N/A'}))
    
    return dict_out

# 1. Movie metadata ETL

In [3]:
# Read everythin as str
metadata_df = pd.read_csv(metadata_file, dtype = 'str')

In [4]:
# Grab only the columns that we need for later analysis
metadata_sub_df = metadata_df[['budget', 'revenue', 'id', 'original_title', 'title']]
metadata_sub_df.head()

,budget,revenue,id,original_title,title
0,30000000,373554033,862,Toy Story,Toy Story
1,65000000,262797249,8844,Jumanji,Jumanji
2,0,0,15602,Grumpier Old Men,Grumpier Old Men
3,16000000,81452156,31357,Waiting to Exhale,Waiting to Exhale
4,0,76578911,11862,Father of the Bride Part II,Father of the Bride Part II


In [5]:
# Flush-out invalid values in the 'budget' field
for record in list(metadata_sub_df['budget'].unique()):
    try:
        float(record)
    except:
        print(f'"{record}" not valid for conversion to numeric.')

"/ff9qCepilowshEtG2GYWwzt2bs4.jpg" not valid for conversion to numeric.
"/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg" not valid for conversion to numeric.
"/zaSf5OG7V8X8gqFvly88zDdRm46.jpg" not valid for conversion to numeric.


In [6]:
# Drop the rows which contain invalid budget values
drop_indices = metadata_sub_df['budget'].loc[metadata_sub_df['budget'].str.match('/')].index
clean_metadata_sub_df = metadata_sub_df.drop(drop_indices, axis = 0)

In [7]:
# Convert to numeric
clean_metadata_sub_df['budget'] = pd.to_numeric(clean_metadata_sub_df['budget'])
clean_metadata_sub_df['revenue'] = pd.to_numeric(clean_metadata_sub_df['revenue'])

# Sanity check
clean_metadata_sub_df.dtypes

budget              int64
revenue           float64
id                 object
original_title     object
title              object
dtype: object

In [8]:
# Write to .csv for loading into a database in Postgres
output_path = os.path.join('Resources','movie_metadata_transformed.csv')
clean_metadata_sub_df.to_csv(output_path, index = False)

# 2. Movie credits ETL

In [9]:
credit_df = pd.read_csv(credit_file)

In [10]:
credit_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [11]:
credit_df.dtypes

cast    object
crew    object
id       int64
dtype: object

In [12]:
dict_list = []
for record in credit_df['crew']:
    dict_list.append(dictify(record))

In [13]:
movie_ids = list(credit_df['id'])

In [14]:
crew_list = []
movie_counter = 0
for crew_data in dict_list: 
    for record in (crew_data):
        try:
            if (record['department'] == 'Writing'):
                crew_list.append({'movie_id':movie_ids[movie_counter],\
                                  'Writer': record['name']})
        except:
            crew_list.append({'movie_id':movie_ids[movie_counter],\
                             'Writer': 'N/A'})
    movie_counter = movie_counter + 1

In [ ]:
movie_writers_df = pd.DataFrame(crew_list)

In [ ]:
output_path = os.path.join('Resources','movie_writers_df.csv')
movie_writers_df.to_csv('movie_writers_df.csv', index = False)